<a href="https://colab.research.google.com/github/cbonnin88/InnovateNext-HR_Project/blob/main/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import polars as pl
import polars.selectors as cs
from datetime import datetime
import plotly.express as px
import gdown as gd
import pandas as pd

In [ ]:
url = 'https://drive.google.com/uc?id=1tUdeNd0_9-itaIgdJtK8_FgWlbDkvxyk'
gd.download(url,'master_df.csv',quiet=True)


df_final = pl.read_csv('master_df.csv')

In [ ]:
df_final.head()

EmployeeID,FirstName,LastName,Department,Role,HireDate,TerminationDate,Gender,Location,Salary,Bonus,StockOptions,PerformanceRating,Promotion,EngagementScore,ManagerRating,Attrition,TenureInDays
str,str,str,str,str,str,str,str,str,i64,f64,i64,i64,str,i64,i64,i64,i64
"""E1023""","""Claude""","""Renard""","""Engineering""","""Senior Accountant""","""2016-01-24""",null,"""Unknown""","""Nantes""",199042,6642.06,1200,3,"""No""",3,6,0,3571
"""E1647""","""Joséphine""","""Riou""","""Finance""","""Senior Accountant""","""2019-02-08""",null,"""Female""","""Bordeaux""",84362,3984.27,200,3,"""No""",9,3,0,2460
"""E1941""","""Stéphane""","""Rocher""","""Sales""","""Analyst""","""2021-06-26""",null,"""Unknown""","""Bordeaux""",232090,10044.19,200,4,"""No""",8,3,0,1591
"""E1695""","""Alix""","""Chauvin""","""Unknown""","""VP of Sales""","""2016-09-21""",null,"""Female""","""Bordeaux""",193920,5380.39,500,null,"""Yes""",3,5,0,3330
"""E1584""","""Vincent""","""Gonzalez""","""Engineering""","""Digital Marketing Analyst""","""2022-08-27""","""2023-12-10""","""Female""","""Paris""",178481,7453.43,500,3,"""No""",6,10,1,470


# **What is the overall attrition Rate ?**

In [ ]:
# Creating an 'AttritionStatus' column
df_final_plots = df_final.with_columns(
    AttritionStatus = pl.when(pl.col('Attrition')== 1).then(pl.lit('Left')).otherwise(pl.lit('Stayed'))
)

In [ ]:
# Calculate overall attrition
overall_attrition_rate = df_final.select(pl.col('Attrition').mean())
print(f'Overall Attrition Rate: {overall_attrition_rate.item() * 100:.0f}%')

Overall Attrition Rate: 31%


In [ ]:
# Overall Attrition Rate by Department
attrition_by_dept = (
    df_final_plots.group_by('Department')
    .agg(
        AttritionRate=(pl.col('Attrition').mean()*100)
    )
    .sort('AttritionRate',descending=True)
)

print('\n--- Attrition Rate by Department ---')
display(attrition_by_dept)


--- Attrition Rate by Department ---


Department,AttritionRate
str,f64
"""Unknown""",38.888889
"""Finance""",33.72093
"""Marketing""",32.432432
"""Sales""",32.103321
"""Human Resources""",30.078125
"""Engineering""",26.041667


# **Attrition Rate by Department**

In [ ]:
fig1 = px.bar(
    attrition_by_dept,
    x='Department',
    y='AttritionRate',
    title='Attrition Rate by Department',
    color='Department',
    labels= {'AttritionRate':'Attrition Rate (%)'},
    text_auto='.0%',
)

fig1.update_traces(
    texttemplate='%{y:.0f}%',

)
fig1.show()

# **Attrition Rate by Role**

In [ ]:
# Filter for roles with at least 5 employees to reduce noise
attrition_by_role = (
    df_final_plots.group_by('Role')
    .agg(
        AttritionRate=(pl.col('Attrition').mean()*100),
        EmployeeCount=pl.col('EmployeeID').count()
    )
    .filter(pl.col('EmployeeCount')>=5)
    .sort('AttritionRate', descending=True)
    .head(10) # Getting the top 10 highest-attrition roles
)

display(attrition_by_role)

Role,AttritionRate,EmployeeCount
str,f64,u32
"""Product Marketing Manager""",63.636364,11
"""SEO Specialist""",55.555556,9
"""Compensation Analyst""",44.444444,18
"""Machine Learning Engineer""",43.478261,23
"""HR Coordinator""",40.909091,22
"""Accountant""",40.740741,27
"""FP&A Manager""",40.0,25
"""HR Data Scientist""",40.0,25
"""Payroll Specialist""",39.285714,28


In [ ]:
fig2=px.bar(
    attrition_by_role,
    x='Role',
    y='AttritionRate',
    title='Top 10 High-Attrition Roles (with >= 5 employees)',
    labels= {'AttritionRate':'Attrition Rate (%)','Role':'Occupation'},
    color='Role',
    color_discrete_sequence=px.colors.qualitative.Pastel
)

fig2.update_traces(
    texttemplate='%{y:.0f}%',
)

fig2.show()

# **Salary Distribution by Attrition**

In [ ]:
fig3= px.box(
    df_final_plots,
    x='AttritionStatus',
    y='Salary',
    color='AttritionStatus',
    title='Salary Distribution by Attrition Status',
    points='all'
)

fig3.show()

# **Performance vs Salary **

In [ ]:
fig4 = px.scatter(
    df_final_plots.drop_nulls(), # Dropping nulls for a clean plot
    x='PerformanceRating',
    y='Salary',
    color='AttritionStatus',
    title='Performance vs Salary (Colored by Attrition)',
    trendline='ols',
    color_discrete_sequence=px.colors.qualitative.Pastel
)

fig4.show()

# **Tenure vs Attrition**

In [ ]:
fig5 = px.box(
    df_final_plots,
    x='AttritionStatus',
    y='TenureInDays',
    color='AttritionStatus',
    title='Tenure Distribution by Attrition Status',
    color_discrete_sequence=px.colors.qualitative.Pastel
)

fig5.show()